In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm
import transformers
import torch
import torch.nn as nn
import random
import os

from torch.nn import functional as F
from transformers import RobertaTokenizer, BertTokenizer
from torch.utils.data import Dataset, DataLoader
# from transformers import BertForSequenceClassification 
from transformers import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [4]:
BATCH_SIZE = 32
MODEL_MAX_LENGTH = 256

In [5]:
def loss_fn(output, target):
    return nn.CrossEntropyLoss()(output, target)

In [6]:
class MyDataSet(Dataset):
  def __init__(self, **kwargs):
    super().__init__()
    pass
    
  def __len__(self):
    pass 

  def __getitem__(self):
    pass

In [7]:
class MyDataSet(Dataset):
  def __init__(self, df, tokenizer, max_length, tk_with = "bert"):
    super().__init__()
    self.df = df
    self.tokenizer = tokenizer
    self.max_length = max_length
    self.target = self.df.loc[:,'label']
    self.tk_with = tk_with
    
  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    text1, text2 = self.df.iloc[index][['premise', 'hypothesis']]

    if self.tk_with == "bert" :
      inputs = self.tokenizer(text1, text2,
                              return_attention_mask = True,
                              add_special_tokens = True,
                              padding = "max_length",
                              return_tensors = "np")
      
      ids = inputs["input_ids"]
      token_type_ids = inputs["token_type_ids"]
      mask = inputs["attention_mask"]
      
      return {
              'ids': ids,
              'mask': mask,
              'token_type_ids': token_type_ids,
              'target': torch.tensor(self.df.iloc[index]["label"], dtype=torch.long)
              }
    if self.tk_with == "xlm-roberta":
      inputs = self.tokenizer(text1, text2,
                              return_attention_mask = True,
                              return_token_type_ids = True,
                              add_special_tokens = True,
                              padding = "max_length",
                              return_tensors = "np")
      
      ids = inputs["input_ids"]
      mask = inputs["attention_mask"]
      token_type_ids = inputs["token_type_ids"]
      
      return {'ids': ids,
              'mask': mask,
              'token_type_ids': token_type_ids,
              'target': torch.tensor(self.df.iloc[index]["label"], dtype=torch.long)
              } 

# dataloader = DataLoader(dataset = dataset, batch_size = BATCH_SIZE)

In [22]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",
                                               model_max_length = MODEL_MAX_LENGTH,
                                              padding_side = "right",
                                              truncation_side = "right",)

In [10]:
df_train = pd.read_csv("/content/drive/MyDrive/Personal Projects/NLP POCs/contradictory-my-dear-watson/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Personal Projects/NLP POCs/contradictory-my-dear-watson/test.csv")

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_train.index.values, 
                                                  df_train.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df_train.label.values)

df_train['data_type'] = ""

df_train.loc[X_train, 'data_type'] = 'train'
df_train.loc[X_val, 'data_type'] = 'val'

In [34]:
def encode_one(x):
  encoded = bert_tokenizer(x['hypothesis'], x['premise'],
                           return_attention_mask = True,
                            add_special_tokens = True,
                            padding = "max_length",
                            truncation = 'longest_first',
                            return_tensors = "np") 
  return encoded['input_ids'], encoded['token_type_ids'], encoded['attention_mask']

In [35]:
df_train[["input_ids","token_type_ids", "attention_mask"]] = df_train.apply(lambda x : encode_one(x), axis = 1, result_type = 'expand')

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [36]:
from torch.utils.data import TensorDataset

In [38]:
filter_dtype_train = df_train.data_type == "train"

input_ids_train = torch.from_numpy(np.stack(df_train.loc[filter_dtype_train,'input_ids'].values))
attention_masks_train = torch.from_numpy(np.stack(df_train.loc[filter_dtype_train,'attention_mask'].values))
labels_train = torch.tensor(df_train.loc[filter_dtype_train,'label'].values)

input_ids_val = torch.from_numpy(np.stack(df_train.loc[~filter_dtype_train,'input_ids'].values))
attention_masks_val = torch.from_numpy(np.stack(df_train.loc[~filter_dtype_train,'attention_mask'].values))
labels_val = torch.tensor(df_train.loc[~filter_dtype_train,'label'].values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [39]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

dataloader_train = DataLoader(dataset_train, 
                              sampler=SequentialSampler(dataset_train), 
                              batch_size=BATCH_SIZE)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=BATCH_SIZE)

In [40]:
df_train = df_train[df_train.lang_abv == "en"].reset_index(drop = True)
df_test = df_test[df_test.lang_abv == "en"].reset_index(drop = True)

In [41]:
df_train.label.value_counts()

0    2427
2    2277
1    2166
Name: label, dtype: int64

In [42]:
epochs = 3

In [57]:
class BERT(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert_model = transformers.BertModel.from_pretrained("bert-base-uncased")
        self.dp1 = nn.Dropout(p=0.1)
        self.ffnn1 = nn.Linear(768, 3)
        
        
    def forward(self,input_ids,attention_mask,token_type_ids, labels):
        _,o2= self.bert_model(input_ids = ids,
                              attention_mask = mask,
                              token_type_ids = token_type_ids, 
                              return_dict=False)
        
        out = self.dp1(o2)
        out = torch.sigmoid((self.ffnn1(out)))
        
        return out
    
model=BERT()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# for param in model.xlm_roberta_model.parameters():
#     param.requires_grad = True

In [47]:
for p,p_x in model.named_parameters():
    if p_x.requires_grad:
      print(p)

bert_model.embeddings.word_embeddings.weight
bert_model.embeddings.position_embeddings.weight
bert_model.embeddings.token_type_embeddings.weight
bert_model.embeddings.LayerNorm.weight
bert_model.embeddings.LayerNorm.bias
bert_model.encoder.layer.0.attention.self.query.weight
bert_model.encoder.layer.0.attention.self.query.bias
bert_model.encoder.layer.0.attention.self.key.weight
bert_model.encoder.layer.0.attention.self.key.bias
bert_model.encoder.layer.0.attention.self.value.weight
bert_model.encoder.layer.0.attention.self.value.bias
bert_model.encoder.layer.0.attention.output.dense.weight
bert_model.encoder.layer.0.attention.output.dense.bias
bert_model.encoder.layer.0.attention.output.LayerNorm.weight
bert_model.encoder.layer.0.attention.output.LayerNorm.bias
bert_model.encoder.layer.0.intermediate.dense.weight
bert_model.encoder.layer.0.intermediate.dense.bias
bert_model.encoder.layer.0.output.dense.weight
bert_model.encoder.layer.0.output.dense.bias
bert_model.encoder.layer.0.outp

In [48]:
SAVE_DIR = "/content/drive/MyDrive/Personal Projects/NLP POCs/contradictory-my-dear-watson/models"
MODEL_NAME = 'BERT_model_v2.0.pth'

In [ ]:
# path = os.path.join(SAVE_DIR, MODEL_NAME)
# model = BERT()
# model = model.load_state_dict(torch.load(path))

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [58]:
model.to(device)

BERT(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [55]:
optimizer= AdamW(model.parameters(), lr=1e-4)

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0].view(-1,MODEL_MAX_LENGTH),
                  'attention_mask': batch[1].view(-1,MODEL_MAX_LENGTH),
                  'labels':         batch[2]
                 }     
        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [51]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0].view(-1,MODEL_MAX_LENGTH),
                  'attention_mask': batch[1].view(-1,MODEL_MAX_LENGTH),
                  "token_type_ids" : None,
                  'labels':         batch[2]
                 }      

        outputs = model(**inputs)
        
        loss = F.cross_entropy(inputs['labels'], outputs['logits'].softmax(dim=1))
        # loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        # scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
model.eval()
    
loss_val_total = 0
predictions, true_vals = [], []

for batch in dataloader_validation:
    
    batch = tuple(b.to(device) for b in batch)
    
    inputs = {'input_ids':      batch[0].view(-1,MODEL_MAX_LENGTH),
              'attention_mask': batch[1].view(-1,MODEL_MAX_LENGTH),
              'labels':         batch[2]
              }     
    with torch.no_grad():        
        outputs = model(**inputs)
        
    loss = outputs[0]
    logits = outputs[1]
    loss_val_total += loss.item()

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_vals.append(label_ids)

loss_val_avg = loss_val_total/len(dataloader_validation) 

predictions = np.concatenate(predictions, axis=0)
true_vals = np.concatenate(true_vals, axis=0)

TypeError: ignored

In [ ]:
sum(predictions.argmax(axis=1) == true_vals) / len(true_vals) * 100

{2}